# Effects of COVID-19 on Select Army Futures Command Installations

#### Author: Peter Flaming

## Notebook Setup

First, let's import a few common modules, ensure MatplotLib plots figures inline and prepare a function to save the figures. We also check that Python 3.5 or later is installed (although Python 2.x may work, it is deprecated so we strongly recommend you use Python 3 instead), as well as Scikit-Learn ≥0.20 and TensorFlow ≥2.0.

In [2]:
# Python ≥3.5 is required
import os
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. Models can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common data science imports
import numpy as np
import pandas as pd

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

from pylab import rcParams
rcParams['figure.figsize'] = 20, 10
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 999

import seaborn as sns

from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import ColumnDataSource, ColorBar
from bokeh.palettes import Spectral6
from bokeh.transform import linear_cmap
from bokeh.plotting import figure
output_notebook()

# Where to save the figures
PROJECT_ROOT_DIR = "."
PROJECT_ID = "COVID-19"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "Images", PROJECT_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

# Create function to save figures
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
    
# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")
warnings.simplefilter('ignore', DeprecationWarning)

Using plaidml.keras.backend backend.


No GPU was detected. Models can be very slow without a GPU.


Loading BokehJS ...

# Get the data

In [9]:
# Download data from URLs
US_Daily_URL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/06-22-2020.csv"
US_Hospitals_URL = "https://raw.githubusercontent.com/Data/Hospitals.csv"
US_Current_URL = "https://covidtracking.com/api/v1/states/current.csv"

daily_raw = pd.read_csv(US_Daily_URL)
hospitals_raw = pd.read_csv(US_Hospitals_URL)
current_raw = pd.read_csv(US_Current_URL)

In [10]:
daily.head()

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3,Testing_Rate,Hospitalization_Rate
0,Alabama,US,2020-06-23 04:33:35,32.3182,-86.9023,30454,841,15974.0,13639.0,1,621.106485,349110.0,2471.0,2.761542,84000001,USA,7120.065835,8.113877
1,Alaska,US,2020-06-23 04:33:35,61.3707,-152.4044,758,12,491.0,255.0,2,103.616319,90824.0,NaN,1.583113,84000002,USA,12415.367476,NaN
2,American Samoa,US,2020-06-23 04:33:35,-14.2710,-170.1320,0,0,NaN,0.0,60,0.000000,174.0,NaN,NaN,16,ASM,312.719038,NaN
3,Arizona,US,2020-06-23 04:33:35,33.7298,-111.4312,54599,1351,7581.0,45667.0,4,750.118462,429790.0,4185.0,2.474404,84000004,USA,5904.749422,7.664976
4,Arkansas,US,2020-06-23 04:33:35,34.9697,-92.3731,16083,227,10793.0,5063.0,5,532.937195,256339.0,1164.0,1.411428,84000005,USA,8494.222952,7.237456


In [11]:
hospitals.head()

,"<!DOCTYPE html><html lang=""en-us""><head><title>Hospitals</title><meta name=""twitter:title"" content=""Hospitals""><meta property=""og:title"" content=""Hospitals""><meta name=""description"" content=""This feature class/shapefile contains locations of Hospitals for 50 US states",Washington D.C.,US territories of Puerto Rico,Guam,American Samoa,Northern Mariana Islands,Palau,and Virgin Islands. The dataset only includes hospital facilities based on data acquired from various state departments or federal sources which has been referenced in the SOURCE field. Hospital facilities which do not occur in these sources will be not present in the database. The source data was available in a variety of formats (pdfs,tables,webpages,etc.) which was cleaned and geocoded and then converted into a spatial database. The database does not contain nursing homes or health centers. Hospitals have been categorized into children,chronic disease,critical access,general acute care,long term care,military,psychiatric,rehabilitation,special,"and women based on the range of the available values from the various sources after removing similarities. In this update 123 additional hospitals were added and 26 additional helipads were identified.""><meta name=""twitter:description"" content=""This feature class/shapefile contains locations of Hospitals for 50 US states",Washington D.C..1,US territories of Puerto Rico.1,Guam.1,American Samoa.1,Northern Mariana Islands.1,Palau.1,and Virgin Islands. The dataset only includes hospital facilities based on data acquired from various state departments or federal sources which has been referenced in the SOURCE field. Hospital facilities which do not occur in these sources will be not present in the database. The source data was available in a variety of formats (pdfs.1,tables.1,webpages.1,etc.) which was cleaned and geocoded and then converted into a spatial database. The database does not contain nursing homes or health centers. Hospitals have been categorized into children.1,chronic disease.1,critical access.1,general acute care.1,long term care.1,military.1,psychiatric.1,rehabilitation.1,special.1,"and women based on the range of the available values from the various sources after removing similarities. In this update 123 additional hospitals were added and 26 additional helipads were identified.""><meta property=""og:description"" content=""This feature class/shapefile contains locations of Hospitals for 50 US states",Washington D.C..2,US territories of Puerto Rico.2,Guam.2,American Samoa.2,Northern Mariana Islands.2,Palau.2,and Virgin Islands. The dataset only includes hospital facilities based on data acquired from various state departments or federal sources which has been referenced in the SOURCE field. Hospital facilities which do not occur in these sources will be not present in the database. The source data was available in a variety of formats (pdfs.2,tables.2,webpages.2,etc.) which was cleaned and geocoded and then converted into a spatial database. The database does not contain nursing homes or health centers. Hospitals have been categorized into children.2,chronic disease.2,critical access.2,general acute care.2,long term care.2,military.2,psychiatric.2,rehabilitation.2,special.2,"and women based on the range of the available values from the various sources after removing similarities. In this update 123 additional hospitals were added and 26 additional helipads were identified.""><meta name=""twitter:image"" content=""https://www.arcgis.com/sharing/rest/content/items/6ac5e325468c4cb9b905f1728d6fbf0f/info/thumbnail/thumbnail1591636820663.png?w=500""><meta property=""og:image"" content=""https://www.arcgis.com/sharing/rest/content/items/6ac5e325468c4cb9b905f1728d6fbf0f/info/thumbnail/thumbnail1591636820663.png?w=500""><meta name=""twitter:card"" content=""summary""><meta property=""og:url"" content=""https://hifld-geoplatform.opendata.arcgis.com/datasets/hospitals?geometry=49.230",-16.829,-80.145,